In [1]:
import random
import heapq
from collections import deque

In [7]:
class Warehouse:
    def __init__(self, seed):
        random.seed(seed)
        self.N = random.randint(5, 10)
        self.M = random.randint(5, 10)
        self.P = random.randint(2, 6)
        self.O = random.randint(1, 10)
        self.start = (0, 0)
        self.packages = []
        self.dropoffs = []
        self.obstacles = set()
        
        occupied = {self.start}
        
        # Generate packages and dropoffs
        for _ in range(self.P):
            while True:
                p = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if p not in occupied:
                    break
            occupied.add(p)
            
            while True:
                d = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if d not in occupied:
                    break
            occupied.add(d)
            
            self.packages.append(p)
            self.dropoffs.append(d)
        
        # Generate obstacles
        for _ in range(self.O):
            while True:
                o = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if o not in occupied:
                    self.obstacles.add(o)
                    occupied.add(o)
                    break

    def display_grid(self):
        grid = [['.' for _ in range(self.M)] for _ in range(self.N)]
        grid[self.start[0]][self.start[1]] = 'S'
        
        for x, y in self.packages:
            grid[x][y] = 'P'
        for x, y in self.dropoffs:
            grid[x][y] = 'D'
        for x, y in self.obstacles:
            grid[x][y] = 'O'
        
        print("Initial Warehouse Configuration:")
        for row in grid:
            print(' '.join(row))
        print()

In [8]:
class Agent:
    def __init__(self, warehouse):
        self.warehouse = warehouse
        self.current_pos = warehouse.start
        self.total_cost = 0
        self.total_reward = 0
        self.penalties = 0
        self.path_taken = [warehouse.start]

    def get_neighbors(self, pos):
        x, y = pos
        return [(x+dx, y+dy) for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)] 
                if 0 <= x+dx < self.warehouse.N and 0 <= y+dy < self.warehouse.M]

    def bfs(self, start, goal):
        queue = deque([[start]])
        visited = set([start])
        
        while queue:
            path = queue.popleft()
            current = path[-1]
            if current == goal:
                return path
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited and neighbor not in self.warehouse.obstacles:
                    visited.add(neighbor)
                    queue.append(path + [neighbor])
        return None

    def dfs(self, start, goal):
        stack = [[start]]
        visited = set([start])
        
        while stack:
            path = stack.pop()
            current = path[-1]
            if current == goal:
                return path
            for neighbor in reversed(self.get_neighbors(current)):
                if neighbor not in visited and neighbor not in self.warehouse.obstacles:
                    visited.add(neighbor)
                    stack.append(path + [neighbor])
        return None

    def ucs(self, start, goal):
        heap = []
        heapq.heappush(heap, (0, [start]))
        visited = {start: 0}
        
        while heap:
            cost, path = heapq.heappop(heap)
            current = path[-1]
            
            if current == goal:
                return path
                
            for neighbor in self.get_neighbors(current):
                if neighbor in self.warehouse.obstacles:
                    continue
                    
                new_cost = cost + 1  # Movement cost = 1 per step
                if neighbor not in visited or new_cost < visited[neighbor]:
                    visited[neighbor] = new_cost
                    heapq.heappush(heap, (new_cost, path + [neighbor]))
        return None

    def find_path(self, start, goal, algorithm):
        if algorithm == 'bfs':
            return self.bfs(start, goal)
        elif algorithm == 'dfs':
            return self.dfs(start, goal)
        elif algorithm == 'ucs':
            return self.ucs(start, goal)
        raise ValueError("Invalid algorithm")

    def deliver_packages(self, algorithm='ucs'):
        for i in range(len(self.warehouse.packages)):
            # Move to package
            pkg_path = self.find_path(self.current_pos, 
                                    self.warehouse.packages[i], 
                                    algorithm)
            if not pkg_path:
                print(f"Package {i+1} unreachable!")
                continue
                
            self.path_taken += pkg_path[1:]
            self.total_cost += len(pkg_path) - 1
            self.current_pos = self.warehouse.packages[i]
            
            # Move to dropoff
            drop_path = self.find_path(self.current_pos, 
                                     self.warehouse.dropoffs[i], 
                                     algorithm)
            if not drop_path:
                print(f"Dropoff {i+1} unreachable!")
                continue
                
            self.path_taken += drop_path[1:]
            self.total_cost += len(drop_path) - 1
            self.current_pos = self.warehouse.dropoffs[i]
            self.total_reward += 10
            
        final_score = self.total_reward - self.total_cost - (self.penalties * 5)
        return {
            'algorithm': algorithm,
            'path': self.path_taken,
            'total_cost': self.total_cost,
            'total_reward': self.total_reward,
            'final_score': final_score
        }

In [10]:
seed = 42
warehouse = Warehouse(seed)
warehouse.display_grid()
for algorithm in ["bfs","dfs","ucs"]:
    print(f"\nResults using {algorithm.upper()}:")

    agent = Agent(warehouse)
    results = agent.deliver_packages(algorithm)

    print(f"Path: {' -> '.join([ str(i) for i in results['path']])}")
    print(f"Total Movement Cost: {results['total_cost']}")
    print(f"Delivery Rewards: {results['total_reward']}")
    print(f"Final Score: {results['final_score']}")


Initial Warehouse Configuration:
S . . . .
. O . . .
D . . . .
. P . . O
. . . . .
. . . . .
. . . . .
. . . . .
P O . O .
O . . D .


Results using BFS:
Path: (0, 0) -> (1, 0) -> (2, 0) -> (3, 0) -> (3, 1) -> (2, 1) -> (2, 0) -> (3, 0) -> (4, 0) -> (5, 0) -> (6, 0) -> (7, 0) -> (8, 0) -> (7, 0) -> (7, 1) -> (7, 2) -> (8, 2) -> (9, 2) -> (9, 3)
Total Movement Cost: 18
Delivery Rewards: 20
Final Score: 2

Results using DFS:
Path: (0, 0) -> (1, 0) -> (2, 0) -> (3, 0) -> (3, 1) -> (2, 1) -> (2, 0) -> (1, 0) -> (0, 0) -> (0, 1) -> (0, 2) -> (1, 2) -> (2, 2) -> (3, 2) -> (4, 2) -> (5, 2) -> (6, 2) -> (7, 2) -> (7, 1) -> (7, 0) -> (8, 0) -> (7, 0) -> (6, 0) -> (5, 0) -> (4, 0) -> (3, 0) -> (2, 0) -> (1, 0) -> (0, 0) -> (0, 1) -> (0, 2) -> (1, 2) -> (2, 2) -> (3, 2) -> (4, 2) -> (5, 2) -> (6, 2) -> (7, 2) -> (8, 2) -> (9, 2) -> (9, 3)
Total Movement Cost: 40
Delivery Rewards: 20
Final Score: -20

Results using UCS:
Path: (0, 0) -> (1, 0) -> (2, 0) -> (2, 1) -> (3, 1) -> (2, 1) -> (2, 0) -> (3